In [ ]:
import pandas
import numpy
import matplotlib.pyplot

In [ ]:
data=pandas.read_csv('/content/sentiment_tweets3.csv')

In [ ]:
data.head(3)

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0


In [ ]:
data.drop('Index',axis=1,inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10314 entries, 0 to 10313
Data columns (total 2 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   message to examine         10314 non-null  object
 1   label (depression result)  10314 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 161.3+ KB


In [ ]:
data['tweet']=data['message to examine']
data.drop('message to examine',axis=1,inplace=True)
data['labels']=data['label (depression result)']
data.drop('label (depression result)',axis=1,inplace=True)

In [ ]:
data.head(5)

,tweet,labels
0,just had a real good moment. i missssssssss hi...,0
1,is reading manga http://plurk.com/p/mzp1e,0
2,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,@lapcat Need to send 'em to my accountant tomo...,0
4,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [ ]:
label2id={0:'NO',1:'YES'}

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,test=train_test_split(data,test_size=0.3)
test,valid=train_test_split(test,test_size=0.3)

In [ ]:
from datasets import Dataset,DatasetDict

In [ ]:
train_data=DatasetDict(
    {'train':Dataset.from_pandas(train,preserve_index=False),
    'test':Dataset.from_pandas(test,preserve_index=False),
    'valid':Dataset.from_pandas(valid,preserve_index=False)
    }
)

In [ ]:
train_data

DatasetDict({
    train: Dataset({
        features: ['tweet', 'labels'],
        num_rows: 7219
    })
    test: Dataset({
        features: ['tweet', 'labels'],
        num_rows: 2166
    })
    valid: Dataset({
        features: ['tweet', 'labels'],
        num_rows: 929
    })
})

In [ ]:
from transformers import AutoTokenizer
token_model='finiteautomata/bertweet-base-sentiment-analysis'
token=AutoTokenizer.from_pretrained(token_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
def tokenize(data):
  return token(data['tweet'],truncation=True,padding=True,max_length=128)

In [ ]:
tokenized_data=train_data.map(tokenize,batched=True)

Map:   0%|          | 0/7219 [00:00<?, ? examples/s]

Map:   0%|          | 0/2166 [00:00<?, ? examples/s]

Map:   0%|          | 0/929 [00:00<?, ? examples/s]

In [ ]:
tokenized_data['train']

Dataset({
    features: ['tweet', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7219
})

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoModel
model_name='finiteautomata/bertweet-base-sentiment-analysis'
model=AutoModel.from_pretrained(model_name)

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoConfig
import torch

In [ ]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
num_labels=2
config=AutoConfig.from_pretrained(model_name,num_labels=num_labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True).to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
model.config

RobertaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "finiteautomata/bertweet-base-sentiment-analysis",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "tokenizer_class": "BertweetTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.47.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 64001
}

In [ ]:
from sklearn.metrics import f1_score,accuracy_score

def compute_metric(pred):
    label=pred.label_ids
    predict=pred.predictions.argmax(-1)

    f1=f1_score(label,predict)
    accuracy=accuracy_score(label,predict)
    # Return a dictionary instead of a set
    return {'f1': f1, 'accuracy': accuracy} #This fixes the issue by returning a dictionary

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
train_args=TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    disable_tqdm=False,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import tqdm
tqdm.tqdm.pandas()

In [ ]:
trainer=Trainer(model=model,args=train_args,train_dataset=tokenized_data['train'],eval_dataset=tokenized_data['valid'],compute_metrics=compute_metric,tokenizer=token)

<ipython-input-30-bd5c678838f0>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(model=model,args=train_args,train_dataset=tokenized_data['train'],eval_dataset=tokenized_data['valid'],compute_metrics=compute_metric,tokenizer=token)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.040200,0.005944,0.997567,0.998924
2,0.003000,0.000060,1.000000,1.000000
3,0.005900,0.000044,1.000000,1.000000
4,0.000300,0.000031,1.000000,1.000000
5,0.002100,0.000027,1.000000,1.000000


TrainOutput(global_step=4515, training_loss=0.007431133812399641, metrics={'train_runtime': 1171.8525, 'train_samples_per_second': 30.802, 'train_steps_per_second': 3.853, 'total_flos': 2374248385804800.0, 'train_loss': 0.007431133812399641, 'epoch': 5.0})

In [33]:
pred=trainer.predict(tokenized_data['test'])

In [34]:
pred.metrics

{'test_loss': 0.0029487176798284054,
 'test_f1': 0.9978902953586498,
 'test_accuracy': 0.9990766389658357,
 'test_runtime': 17.2037,
 'test_samples_per_second': 125.903,
 'test_steps_per_second': 15.752}

{'tweet': 'home... so happy for my friends who have next week off ', 'labels': 0, 'input_ids': [101, 2188, 1012, 1012, 1012, 2061, 3407, 2005, 2026, 2814, 2040, 2031, 2279, 2733, 2125, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [ ]:
data['tweet'].fillna('just nothing',inplace=True)
data.labels.fillna('2',inplace=True)

<ipython-input-58-e2bd3a9723cb>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['tweet'].fillna('just nothing',inplace=True)
<ipython-input-58-e2bd3a9723cb>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10314 entries, 0 to 10313
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   10314 non-null  object
 1   labels  10314 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 161.3+ KB
